In [2]:
## Bibliotecas e Funções

#!pip install xgboost
#!pip install lightgbm
#!pip install catboost

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score, classification_report, mean_squared_error, r2_score
#from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn import metrics
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.pipeline import make_pipeline

from sklearn.inspection import permutation_importance

from sklearn.compose import make_column_transformer
from sklearn.preprocessing import  OneHotEncoder

from sklearn.metrics import ConfusionMatrixDisplay

# Normalização de dados
from sklearn.preprocessing import StandardScaler

# Algoritmos de aprendizagem
import logging
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier, BaggingClassifier, StackingClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
#from catboost import CatBoostClassifier

#Redes Neurais
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline


import warnings
warnings.filterwarnings("ignore", category=UserWarning)
pd.options.mode.copy_on_write = True # Allow re-write on variable
warnings.filterwarnings("ignore", category=UserWarning) # Ignore warning
warnings.filterwarnings("ignore", category=FutureWarning) # Ignore warning
warnings.filterwarnings("ignore", category=UserWarning) # Ignore warning
warnings.filterwarnings("ignore", category=RuntimeWarning) # Ignore warning
warnings.filterwarnings("ignore", category=DeprecationWarning) # Ignore warning
#sns.set_style('darkgrid') # Seaborn style
warnings.filterwarnings('ignore') # Ignore warnings
pd.set_option('display.max_columns', None) # Setting this option will print all collumns of a dataframe
pd.set_option('display.max_colwidth', None) # Setting this option will print all of the data in a feature

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
#!pip install tpot

## Carregar Dados

#!pip install openpyxl
#!pip install gdown
from google.colab import files
from google.colab import drive
#drive.mount('/content/drive')
import gdown

# ID do arquivo Google Drive
file_id = '1rFf4n9m4eJCUhkjde5di5jrTEkrR22VT'
url = f'https://drive.google.com/uc?id={file_id}'
output = 'arquivo.csv'
gdown.download(url, output, quiet=False)
#dados = pd.read_csv(output)
dados = pd.read_csv(output, encoding='latin-1')

#dados = dados.drop(columns=['Date'])
#dados.info()

df = dados.copy()

# Remover linhas com dados nulos
df = df.dropna()

df.rename(columns={'h_perm_front': 'tempofronteira',
                   'anoModelo': 'anomodelo',
                   'dataEmissaoCrv': 'dataemissaocrv',
                   'descricaoTipoDocumentoProprietario': 'tipodocumentoproprietario',
                   'municipioEmplacamento': 'municipioemplacamento',
                   'ufEmplacamento': 'ufemplacamento',
                   'descricao': 'consulta_res',
                   'ilicito': 'target'
                   }, inplace=True)

df[['marca', 'modelo']] = df['marcaModelo'].str.split('/', expand=True)

# Converter para data
df['dataemissaocrv'] = pd.to_datetime(df['dataemissaocrv'], unit='ms')

#Adicionar essa coluna gera uma colinariedade com a coluna dataemissaocrv
#fixed_date = pd.to_datetime('2024-10-16')
#df['tempocrv'] = (fixed_date - df['dataemissaocrv']).dt.days

df['dataemissaocrv'] = df['dataemissaocrv'].dt.year

df['anomodelo'] = df['anomodelo'].astype(int)

df['unicodono'] = df['dataemissaocrv'] <= df['anomodelo']

df['tempofronteira'] = df['tempofronteira'].astype(int)
df['anomodelo'] = df['anomodelo'].astype(int)
df['dataemissaocrv'] = df['dataemissaocrv'].astype(int)

# Coluna consulta - substituir valores por RES1, RES2, etc
valores_unicos = df['consulta_res'].unique()
codigo_dict = {valor: f'RES{i+1}' for i, valor in enumerate(valores_unicos)}
df['consulta_res'] = df['consulta_res'].map(codigo_dict)

for chave, valor in codigo_dict.items():
    print(f"{valor}: {chave}")

# Apagar colunas não utlizadas mais
df = df.drop('placa', axis=1)
df = df.drop('marcaModelo', axis=1)
#df = df.drop('consulta', axis=1)
#df = df.drop('cor', axis=1)

# Coluna cor - substituir as cores por números de acordo com a intensidade da cor,
              # onde BRANCA = 10 e PRETA = 0
"""
peso_cor = {'AMARELA': 4,'AZUL': 3,'BEGE': 3,'BRANCA': 10,'CINZA': 5,
            'DOURADA': 6,'FANTASIA': 2,'MARROM': 2,'PRATA': 8,'PRETA': 0,
            'VERDE': 5,'VERMELHA': 4,
}
"""
#df['cor'] = df['cor'].map(peso_cor)
#df_pesos = pd.DataFrame(list(peso_cor.items()), columns=['Cor', 'Peso'])
#print("Tabela com pesos atribuídos:")
#print(df)
#print("\nTabela de cores e pesos:")
#print(df_pesos)


#print("")
print(df.info())
#print(df.head())

Downloading...
From: https://drive.google.com/uc?id=1rFf4n9m4eJCUhkjde5di5jrTEkrR22VT
To: /content/arquivo.csv
100%|██████████| 141k/141k [00:00<00:00, 24.0MB/s]

RES1: Não há restrições para este veículo no DENATRAN BASE 24h/7d.
RES2: Veículo com restrição judicial no sistema DENATRAN BASE 24h/7d!
RES3: Veículo com restrição na Receita Federal no sistema DENATRAN BASE 24h/7d!
RES4: Veículo comunicação de venda no sistema DENATRAN BASE 24h/7d!
RES5: Veículo com indicador de Recall ativo no sistema DENATRAN BASE 24h/7d!
RES6: Veículo com registro de Roubo/Furto no sistema DENATRAN BASE 24h/7d!
<class 'pandas.core.frame.DataFrame'>
Index: 745 entries, 0 to 745
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   tempofronteira             745 non-null    int64 
 1   anomodelo                  745 non-null    int64 
 2   categoria                  745 non-null    object
 3   cor                        745 non-null    object
 4   dataemissaocrv             745 non-null    int64 
 5   tipodocumentoproprietario  745 non-null    object
 6   municipioemplacame

In [ ]:
!pip install tpot

In [8]:
import pandas as pd
import numpy as np
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder  # Import LabelEncoder

# Tratar valores ausentes e eliminar colunas desnecessárias
#df = df.drop(columns=['descricao'])

# Selecionar apenas colunas numéricas para calcular a mediana
# Use np.number directly instead of pd.np.number
numeric_cols = df.select_dtypes(include=np.number).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# Convert categorical features to numerical using Label Encoding
# Create a LabelEncoder object
label_encoder = LabelEncoder()

# Get a list of categorical columns (object type)
categorical_cols = df.select_dtypes(include=['object']).columns

# Apply Label Encoding to each categorical column
for col in categorical_cols:
    df[col] = label_encoder.fit_transform(df[col])

# Dividir as variáveis explicativas (X) da variável alvo (y)
X = df.drop(columns=['target'])  # Todas as colunas exceto 'ilicito'
y = df['target']  # A coluna alvo

# Separar os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Instanciar o TPOTClassifier
tpot = TPOTClassifier(verbosity=2, generations=30, population_size=300, random_state=42)

# Treinar o modelo
tpot.fit(X_train, y_train)

# Fazer previsões
y_pred = tpot.predict(X_test)

# Avaliar o modelo
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-Score: {f1:.4f}')

# Exportar o pipeline gerado
tpot.export('best_pipeline.py')

Optimization Progress:   0%|          | 0/9300 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.850672268907563

Generation 2 - Current best internal CV score: 0.8507142857142856

Generation 3 - Current best internal CV score: 0.8507142857142856

Generation 4 - Current best internal CV score: 0.8507142857142856

Generation 5 - Current best internal CV score: 0.8507142857142856

Generation 6 - Current best internal CV score: 0.8507142857142856

Generation 7 - Current best internal CV score: 0.8624229691876749

Generation 8 - Current best internal CV score: 0.8624229691876749

Generation 9 - Current best internal CV score: 0.8624229691876749

Generation 10 - Current best internal CV score: 0.8624229691876749

Generation 11 - Current best internal CV score: 0.8624229691876749

Generation 12 - Current best internal CV score: 0.8624229691876749

Generation 13 - Current best internal CV score: 0.8624229691876749

Generation 14 - Current best internal CV score: 0.8624229691876749

Generation 15 - Current best internal CV score: 0.8624229

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from datetime import datetime
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import warnings

# Custom Transformer
class DiasUltimoCrvTransformer(BaseEstimator, TransformerMixin):
    def _init_(self, feature_name='dataEmissaoCrv'):
        self.feature_name = feature_name

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_ = X.copy()  # Create a copy to avoid modifying the original DataFrame
        X_['diasUltimoCrv'] = X_[self.feature_name].apply(self.calcular_dias_ultimo_crv)
        return X_

    def calcular_dias_ultimo_crv(self, data_milisegundos):
        data_emissao = datetime.fromtimestamp(data_milisegundos / 1000)
        dias_diferenca = (datetime.now() - data_emissao).days
        return dias_diferenca

# Importando Dados
# Carregar Dados

#!pip install openpyxl
#!pip install gdown
from google.colab import files
from google.colab import drive
#drive.mount('/content/drive')
import gdown

# ID do arquivo Google Drive
file_id = '1rFf4n9m4eJCUhkjde5di5jrTEkrR22VT'
url = f'https://drive.google.com/uc?id={file_id}'
output = 'arquivo.csv'
gdown.download(url, output, quiet=False)
#dados = pd.read_csv(output)
dados = pd.read_csv(output, encoding='latin-1')

#dados = dados.drop(columns=['Date'])
dados.info()

df = dados.copy()
df = df.dropna()
# Preparando os dados
X = df.drop(['placa', 'ilicito'], axis=1)
y = df['ilicito'].astype(int)

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Definindo as colunas numéricas e categóricas
numerical_features = X.select_dtypes(include=['number']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Pré-processador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
        #('dias_crv', DiasUltimoCrvTransformer(), ['dataEmissaoCrv'])
    ])

# Definindo os modelos e seus hiperparâmetros
models = {
    'Logistic Regression': {
        'model': Pipeline([('preprocessor', preprocessor), ('classifier', LogisticRegression(max_iter=10000, solver="liblinear"))]),
        'params': {
            'classifier__C': [0.001, 0.1, 1, 10, 100],
            'classifier__penalty': ['l1', 'l2']
        }
    },
    'Decision Tree': {
        'model': Pipeline([('preprocessor', preprocessor), ('classifier', DecisionTreeClassifier())]),
        'params': {
            'classifier__max_depth': [3, 7, 20],
            'classifier__min_samples_split': [2, 10, 30]
        }
    },
    'Random Forest': {
        'model': Pipeline([('preprocessor', preprocessor), ('classifier', RandomForestClassifier())]),
        'params': {
            'classifier__n_estimators': [50, 200, 500, 900],
            'classifier__max_depth': [3, 7, 15, 30],
            'classifier__min_samples_split': [2, 5, 15, 50],
            'classifier__min_samples_leaf': [2, 5, 10],
            'classifier__max_features': ['log2', 'sqrt', None]
        }
    },
    'Gradient Boosting': {
        'model': Pipeline([('preprocessor', preprocessor), ('classifier', GradientBoostingClassifier())]),
        'params': {
            'classifier__n_estimators': [50, 200],
            'classifier__learning_rate': [0.1, 0.1, 0.001]
        }
    },
    'XGBoost': {
        'model': Pipeline([('preprocessor', preprocessor), ('classifier', XGBClassifier(objective='binary:logistic', random_state=42,use_label_encoder=False, device='cuda'))]),
        'params': {
            'classifier__eta': [0.01, 0.05, 0.1],
            'classifier__gamma': [0.05, 0.5, 1.0],
            'classifier__max_depth': [3, 9, 25],
            'classifier__min_child_weight': [1, 5, 10],
            'classifier__subsample': [0.6, 0.8, 1],
            'classifier__colsample_bytree': [0.6, 0.8, 1],
            'classifier__lambda': [0.01, 1.0],
            'classifier__alpha': [0, 0.5, 0.1]
        }
    },
   # 'SVM': {
   #     'model': Pipeline([('preprocessor', preprocessor), ('classifier', SVC())]),
   #     'params': {
   #         'classifier__C': [0.01, 1, 5],
    #        'classifier__kernel': ['linear', 'rbf'],
    #        'classifier__gamma': ['scale', 'auto'],
   #         'classifier__class_weight': ['balanced']
    #    }
   #},
    'K-Nearest Neighbors': {
        'model': Pipeline([('preprocessor', preprocessor), ('classifier', KNeighborsClassifier())]),
        'params': {
            'classifier__n_neighbors': [2, 5, 7, 16],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__p': [1, 2]  # Manhattan e Euclidean distances
        }
    }
}

# Executando GridSearch para cada modelo
results = []
for model_name, model_data in models.items():
    warnings.warn(f"Iniciando GridSearch para o modelo: {model_name}")
    grid_search = GridSearchCV(model_data['model'], model_data['params'], cv=3, scoring='accuracy', error_score='raise')
    grid_search.fit(X_train, y_train)
    y_pred = grid_search.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    results.append({
        'model': model_name,
        'best_params': str(grid_search.best_params_),
        'accuracy': accuracy
    })
    warnings.warn(f"Finalizado: {model_name} com acurácia de {accuracy}")

# Comparando os resultados
results_df = pd.DataFrame(results)
warnings.warn(f"Resultados:\n{results_df}")
output_table_1 = results_df

Downloading...
From: https://drive.google.com/uc?id=1rFf4n9m4eJCUhkjde5di5jrTEkrR22VT
To: /content/arquivo.csv
100%|██████████| 141k/141k [00:00<00:00, 47.4MB/s]
<ipython-input-6-5b0213606b35>:146: UserWarning: Iniciando GridSearch para o modelo: Logistic Regression
  warnings.warn(f"Iniciando GridSearch para o modelo: {model_name}")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 746 entries, 0 to 745
Data columns (total 13 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   placa                               746 non-null    object 
 1   h_perm_front                        746 non-null    float64
 2   anoModelo                           745 non-null    float64
 3   categoria                           745 non-null    object 
 4   cor                                 745 non-null    object 
 5   dataEmissaoCrv                      745 non-null    float64
 6   descricaoTipoDocumentoProprietario  745 non-null    object 
 7   marcaModelo                         745 non-null    object 
 8   municipioEmplacamento               745 non-null    object 
 9   tipo                                745 non-null    object 
 10  ufEmplacamento                      745 non-null    object 
 11  descricao                           746 non-n

A saída de streaming foi truncada nas últimas 5000 linhas.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [02:14:19] WARNING: /workspace/src/context.cc:43: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [02:14:19] WARNING: /workspace/src/context.cc:196: XGBoost is not compiled with CUDA support.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [02:14:19] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [02:14:19] WARNING: /workspace/src/context.cc:43: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [02